# <인구수를 고려한 서울시 공유자전거 대여소 설치 지역 선정을 위한 데이터분석>

## * 맷플롯립 시각화를 위한 글꼴 설치

* 맷플롯립 시각화를 위한 글꼴 설치
* 실행후, [런타임]-[세션 다시 시작] 후 다음 코드부터 실행

In [ ]:
#맷플롯립 시각화를 위한 글꼴 설치
#실행후, [런타임]-[세션 다시 시작] 후 다음 코드부터 실행

!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

## *라이브러리 불러오기 및 글꼴 적용하기(음수부호 오류포함)

In [ ]:
# 필요한 라이브러리 불러오기
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.linear_model import LinearRegression

In [ ]:
# 나눔고딕 적용하기
plt.rcParams['font.family'] = 'NanumGothic'

# 음수부호 오류 처리하기
plt.rcParams['axes.unicode_minus'] = False

## 1. 자치구별 공유자전거 관련 데이터 전처리 및 데이터 파악

### 1-1. 데이터 전처리

In [ ]:
#데이터 프레임의 변수를 bike로 지정
bike = pd.read_csv('bike_seoul.csv')

#bike 데이터프레임의 가장 상위 데이터 5개만 출력
bike.head()

In [ ]:
bike.isnull().sum()

In [ ]:
# 총 따릉이 수 및 총 대여소 수 컬럼 생성 후 추가
# 총 따릉이 수 = LCD형 따릉이 + QR형 따릉이
# 총 대여소 수 = LCD_대여소 + QR_대여소

bike['총 따릉이 수']=bike['LCD형 따릉이']+bike['QR형 따릉이']
bike['총 대여소 수']=bike['LCD_대여소']+bike['QR_대여소']

In [ ]:
bike.head()

### 1-2. 데이터 파악

In [ ]:
# '총 따릉이 수'가 가장 많은 순서대로 상위 데이터 5개만 출력

bike.sort_values('총 따릉이 수',ascending=False).head()

In [ ]:
# '총 대여소 수'가 가장 많은 순서대로 상위 데이터 5개만 출력
# sort_values()를 활용한다.
# 내림차순으로 정렬할 때에는 ascending=False를 추가한다.

bike.sort_values('총 대여소 수',ascending= False).head()

## 2. 자치구별 인구수 관련 데이터 전처리 및 데이터 파악

### 2-1. 데이터 전처리

In [ ]:
#데이터 프레임의 변수를 population로 지정
population = pd.read_csv('population_seoul.csv')

#population 데이터프레임의 가장 상위 데이터 5개만 출력
population.head()

In [ ]:
population.isnull().sum()

In [ ]:
# 컬럼별 이름 수정하기
# 원본을 수정할 때에는 inplace=True를 사용한다.

population.rename(columns={'구분':'자치구',
                     '계':'총 인구수',
                     '계_한국인':'한국인',
                     '계_외국인':'외국인',
                     '65세이상고령자':'고령자'},inplace=True)

In [ ]:
population.head()

### 2-2. 데이터 파악

In [ ]:
# '총 인구수'가 가장 많은 순서대로 상위 데이터 5개만 출력

population.sort_values('총 인구수',ascending=False).head()

## 3. 공유자전거 데이터와 인구수 데이터 합치기

In [ ]:
df_result = pd.merge(bike,population,on='자치구').set_index('자치구')

In [ ]:
#데이터가 잘 합쳐졌는지 확인
df_result.head()

## 4. 데이터 분석 및 시각화

In [ ]:
# 서울시 자치구별 따릉이 대여소 수 시각화 (가로 막대 그래프 사용)

plt.figure(figsize=(10,10))

df_result_sorted = df_result.sort_values('총 대여소 수')

plt.barh(df_result_sorted.index, df_result_sorted['총 대여소 수'],color='r')

plt.title('서울시 자치구별 따릉이 대여소 수')
plt.xlabel('총 대여소 수')
plt.ylabel('자치구별')
plt.grid(True)

plt.show()

In [ ]:
# 서울시 자치구별 인구수 시각화 (가로 막대 그래프 사용)

plt.figure(figsize=(10,10))

df_result_sorted = df_result.sort_values('총 인구수')

plt.barh(df_result_sorted.index, df_result_sorted['총 인구수'],color='r')

plt.title('서울시 자치구별 총 인구수')
plt.xlabel('총 인구수')
plt.ylabel('자치구별')
plt.grid(True)

plt.show()

In [ ]:
#'총 인구수' 변화에 대한 '총 대여소 수' 상관관계 분석
df_result[['총 인구수','총 대여소 수']].corr()

In [ ]:
#회귀분석

plt.figure(figsize=(10,10))

x = np.array(df_result['총 인구수']).reshape(-1, 1)
y = np.array(df_result['총 대여소 수']).reshape(-1, 1)

model = LinearRegression().fit(x,y)

plt.scatter(df_result['총 인구수'],df_result['총 대여소 수'],color='orange')

pred=model.predict(x)
plt.plot(x,pred,color='r')

plt.title('자치구별 총 인구수에 따른 대여소 수 회귀분석')
plt.xlabel('자치구별 총 인구수')
plt.ylabel('자치구별 총 대여소 수')

plt.grid(True)

plt.show()

In [ ]:
#잔차 구하기

x = np.array(df_result['총 인구수']).reshape(-1, 1)
y = np.array(df_result['총 대여소 수']).reshape(-1, 1)

model = LinearRegression().fit(x, y)

y_pred = model.predict(x) #y_perd : 예측값
residuals = y - y_pred

# 잔차를 데이터프레임에 추가
df_result['잔차'] = residuals

df_result['잔차']

In [ ]:
#잔차 정렬하여 출력하기
df_result.sort_values('잔차')

## * 모델 성능 평가하기

In [ ]:
x = np.array(df_result['총 인구수']).reshape(-1,1)
y = np.array(df_result['총 대여소 수']).reshape(-1,1)

model = LinearRegression().fit(x, y)

pred = model.predict(x)

residuals = y - pred

plt.scatter(pred, residuals)

plt.title('잔차 시각화를 통한 모델 성능평가')
plt.xlabel('예측값')
plt.ylabel('잔차')

x_min = pred.min() - 10
x_max = pred.max() + 10

plt.hlines(y=0, xmin=x_min, xmax=x_max, color='red')

plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

x = np.array(df_result['총 인구수']).reshape(-1,1)
y = np.array(df_result['총 대여소 수']).reshape(-1,1)

model =LinearRegression().fit(x,y)
print('평균제곱오차:',mean_squared_error(y,model.predict(x)))
print('결정계수:',r2_score(y,model.predict(x)))